In [ ]:
!pip install yt-dlp requests

import os
import json
import glob
import requests
from urllib.parse import quote
import yt_dlp  # Change this line from youtube_dl to yt_dlp
import re

def clean_filename(filename):
    """Clean filename to remove invalid characters"""
    return re.sub(r'[\\/*?:"<>|]', "", filename)

def create_directory(directory):
    """Create directory if it doesn't exist"""
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

def extract_song_info(json_path):
    """Extract song name and author from a JSON file"""
    try:
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            if 'song' in data and 'title' in data['song'] and 'artist' in data['song']:
                return {
                    'name': data['song']['title'],
                    'author': data['song']['artist']
                }
            else:
                print(f"Missing required fields in {json_path}")
                return None
    except Exception as e:
        print(f"Error reading {json_path}: {e}")
        return None

def search_music(song_name, artist):
    """Search for music using YouTube as a fallback source"""
    search_query = f"{song_name} {artist} audio"

    # yt-dlp options for downloading audio only
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True,
        'no_warnings': True,
        'default_search': 'ytsearch',
        'noplaylist': True,
    }

    try:
        # Search YouTube for the song
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:  # Change this line
            info = ydl.extract_info(f"ytsearch1:{search_query}", download=False)
            if 'entries' in info and info['entries']:
                return info['entries'][0]['webpage_url']
            else:
                return None
    except Exception as e:
        print(f"Error searching for {search_query}: {e}")
        return None

def download_song(url, output_path):
    """Download song from URL to output path"""
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': output_path,
        'quiet': True,
        'no_warnings': True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:  # Change this line
            ydl.download([url])
        return True
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return False

def main():
    # Set up paths
    storage_dir = 'Storage'
    texts_dir = os.path.join(storage_dir, 'temp_texts')
    music_dir = os.path.join(storage_dir, 'music_library')

    # Create music library directory if it doesn't exist
    create_directory(music_dir)

    # Get all JSON files sorted by number
    json_pattern = os.path.join(texts_dir, 'book_*.json')
    json_files = sorted(glob.glob(json_pattern), key=lambda x: int(re.search(r'book_(\d+)\.json', x).group(1)))

    if not json_files:
        print(f"No JSON files found in {texts_dir}")
        return

    print(f"Found {len(json_files)} JSON files")

    # Process each JSON file
    for i, json_path in enumerate(json_files):
        book_num = re.search(r'book_(\d+)\.json', json_path).group(1)
        song_num = book_num  # Use same numbering as book files

        print(f"Processing {json_path}...")

        # Extract song info
        song_info = extract_song_info(json_path)
        if not song_info:
            print(f"Skipping {json_path} due to missing song info")
            continue

        song_name = song_info['name']
        artist = song_info['author']

        print(f"Found song: '{song_name}' by {artist}")

        # Define output filename
        output_file = os.path.join(music_dir, f"song_{song_num}")

        # Search for and download the song
        url = search_music(song_name, artist)
        if url:
            print(f"Downloading '{song_name}' by {artist}...")
            if download_song(url, output_file):
                print(f"Successfully downloaded to {output_file}.mp3")
            else:
                print(f"Failed to download '{song_name}'")
        else:
            print(f"Could not find '{song_name}' by {artist}")

if __name__ == "__main__":
    main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.7 MB/s eta 0:00:00
Found 5 JSON files
Processing Storage/temp_texts/book_1.json...
Found song: 'Interstellar Overdrive' by Pink Floyd
Successfully downloaded to Storage/music_library/song_1.mp3
Processing Storage/temp_texts/book_2.json...
Found song: 'The Ecstasy of Gold' by Ennio Morricone
Successfully downloaded to Storage/music_library/song_2.mp3
Processing Storage/temp_texts/book_3.json...
Found song: 'Clair de Lune' by Claude Debussy
Successfully downloaded to Storage/music_library/song_3.mp3
Processing Storage/temp_texts/book_4.json...
Found song: 'bury a friend' by Billie Eilish
